In [2]:
from langchain_community.graphs import Neo4jGraph
graph=Neo4jGraph(
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
)

In [3]:
graph

In [13]:
from langchain_groq import ChatGroq

llm=ChatGroq(groq_api_key=",model_name="Gemma2-9b-It")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x712d2a693d30>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x712d2a692b60>, model_name='Gemma2-9b-It', groq_api_key=SecretStr('**********'))

In [5]:
from langchain_core.documents import Document
text="""
Elon Reeve Musk (born June 28, 1971) is a businessman and investor known for his key roles in space
company SpaceX and automotive company Tesla, Inc. Other involvements include ownership of X Corp.,
formerly Twitter, and his role in the founding of The Boring Company, xAI, Neuralink and OpenAI.
He is one of the wealthiest people in the world; as of July 2024, Forbes estimates his net worth to be
US$221 billion.Musk was born in Pretoria to Maye and engineer Errol Musk, and briefly attended
the University of Pretoria before immigrating to Canada at age 18, acquiring citizenship through
his Canadian-born mother. Two years later, he matriculated at Queen's University at Kingston in Canada.
Musk later transferred to the University of Pennsylvania and received bachelor's degrees in economics
 and physics. He moved to California in 1995 to attend Stanford University, but dropped out after
  two days and, with his brother Kimbal, co-founded online city guide software company Zip2.
 """
documents=[Document(page_content=text)]
documents

[Document(page_content="\nElon Reeve Musk (born June 28, 1971) is a businessman and investor known for his key roles in space\ncompany SpaceX and automotive company Tesla, Inc. Other involvements include ownership of X Corp.,\nformerly Twitter, and his role in the founding of The Boring Company, xAI, Neuralink and OpenAI.\nHe is one of the wealthiest people in the world; as of July 2024, Forbes estimates his net worth to be\nUS$221 billion.Musk was born in Pretoria to Maye and engineer Errol Musk, and briefly attended\nthe University of Pretoria before immigrating to Canada at age 18, acquiring citizenship through\nhis Canadian-born mother. Two years later, he matriculated at Queen's University at Kingston in Canada.\nMusk later transferred to the University of Pennsylvania and received bachelor's degrees in economics\n and physics. He moved to California in 1995 to attend Stanford University, but dropped out after\n  two days and, with his brother Kimbal, co-founded online city guide 

In [6]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
llm_transformer=LLMGraphTransformer(llm=llm)

In [7]:
graph_documents=llm_transformer.convert_to_graph_documents(documents)

In [11]:
from langchain.chains import GraphCypherQAChain
chain=GraphCypherQAChain.from_llm(llm=llm,graph=graph_documents,verbose=True)
chain

AttributeError: 'list' object has no attribute 'get_structured_schema'

In [12]:
### Load the dataset of movie

movie_query="""
LOAD CSV WITH HEADERS FROM
'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv' as row

MERGE(m:Movie{id:row.movieId})
SET m.released = date(row.released),
    m.title = row.title,
    m.imdbRating = toFloat(row.imdbRating)
FOREACH (director in split(row.director, '|') |
    MERGE (p:Person {name:trim(director)})
    MERGE (p)-[:DIRECTED]->(m))
FOREACH (actor in split(row.actors, '|') |
    MERGE (p:Person {name:trim(actor)})
    MERGE (p)-[:ACTED_IN]->(m))
FOREACH (genre in split(row.genres, '|') |
    MERGE (g:Genre {name:trim(genre)})
    MERGE (m)-[:IN_GENRE]->(g))
"""

In [13]:
graph.query(movie_query)

[]

In [14]:
graph.refresh_schema()
print(graph.schema)

Node properties:
Movie {id: STRING, released: DATE, title: STRING, imdbRating: FLOAT}
Person {name: STRING}
Genre {name: STRING}
Relationship properties:

The relationships:
(:Movie)-[:IN_GENRE]->(:Genre)
(:Person)-[:DIRECTED]->(:Movie)
(:Person)-[:ACTED_IN]->(:Movie)


In [15]:
from langchain.chains import GraphCypherQAChain
chain=GraphCypherQAChain.from_llm(llm=llm,graph=graph,verbose=True)
chain

GraphCypherQAChain(verbose=True, graph=<langchain_community.graphs.neo4j_graph.Neo4jGraph object at 0x74d3ac907f70>, cypher_generation_chain=LLMChain(prompt=PromptTemplate(input_variables=['question', 'schema'], template='Task:Generate Cypher statement to query a graph database.\nInstructions:\nUse only the provided relationship types and properties in the schema.\nDo not use any other relationship types or properties that are not provided.\nSchema:\n{schema}\nNote: Do not include any explanations or apologies in your responses.\nDo not respond to any questions that might ask anything else than for you to construct a Cypher statement.\nDo not include any text except the generated Cypher statement.\n\nThe question is:\n{question}'), llm=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x74d357371060>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x74d3573703a0>, model_name='Gemma2-9b-It', groq_api_key=SecretStr('**********'))), qa_chain=

In [16]:
response=chain.invoke({"query":"Who was the director of the moview GoldenEye"})

response
response=chain.invoke({"query":"Who was the director of the moview GoldenEye"})

response




> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie {title: "GoldenEye"})-[:DIRECTED]->(p:Person) RETURN p.name 

Full Context:
[]

> Finished chain.


{'query': 'Who was the director of the moview GoldenEye',
 'result': "I don't know the answer. \n"}

Lang Graph with Pdf

In [20]:
from langchain.document_loaders import PyPDFLoader

def load_pdf_document(pdf_path):
    loader = PyPDFLoader(pdf_path)
    documents = loader.load()
    return documents

pdf_path = 'data/solar.pdf'
documents = load_pdf_document(pdf_path)


Ignoring wrong pointing object 23 0 (offset 0)
Ignoring wrong pointing object 67 0 (offset 0)
Ignoring wrong pointing object 127 0 (offset 0)
Ignoring wrong pointing object 130 0 (offset 0)
Ignoring wrong pointing object 186 0 (offset 0)
Ignoring wrong pointing object 198 0 (offset 0)
Ignoring wrong pointing object 203 0 (offset 0)
Ignoring wrong pointing object 218 0 (offset 0)
Ignoring wrong pointing object 223 0 (offset 0)
Ignoring wrong pointing object 232 0 (offset 0)
Ignoring wrong pointing object 241 0 (offset 0)
Ignoring wrong pointing object 250 0 (offset 0)
Ignoring wrong pointing object 259 0 (offset 0)
Ignoring wrong pointing object 269 0 (offset 0)


In [22]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [23]:
splitter=RecursiveCharacterTextSplitter(chunk_size=100,chunk_overlap=50)

In [24]:
docs=splitter.split_documents(documents)

In [ ]:
fr

In [1]:
from neo4j import GraphDatabase

In [4]:
driver = GraphDatabase.driver(NEO4J_URI, auth=("neo4j", NEO4J_PASSWORD))

In [20]:
print(type(graph_documents))  # Should be <class 'dict'>
print(graph_documents)  # Inspect the content to confirm it's a dictionary with 'nodes' and 'relationships'


<class 'list'>
[GraphDocument(nodes=[Node(id='Elon Reeve Musk', type='Person'), Node(id='Maye', type='Person'), Node(id='Errol Musk', type='Person'), Node(id='University Of Pretoria', type='Educational_institution'), Node(id='Canada', type='Country'), Node(id="Queen'S University", type='Educational_institution'), Node(id='University Of Pennsylvania', type='Educational_institution'), Node(id='California', type='Region'), Node(id='Stanford University', type='Educational_institution'), Node(id='Kimbal', type='Person'), Node(id='Zip2', type='Company'), Node(id='Spacex', type='Company'), Node(id='Tesla, Inc.', type='Company'), Node(id='X Corp.', type='Company'), Node(id='The Boring Company', type='Company'), Node(id='Xai', type='Company'), Node(id='Neuralink', type='Company'), Node(id='Openai', type='Company'), Node(id='Forbes', type='Organization')], relationships=[Relationship(source=Node(id='Elon Reeve Musk', type='Person'), target=Node(id='Maye', type='Person'), type='PARENT'), Relation

In [7]:
def add_graph_to_neo4j(graph_documents):
    with driver.session() as session:
        # Check if the input is a list with at least one GraphDocument
        if isinstance(graph_documents, list) and len(graph_documents) > 0:
            # Access the first GraphDocument in the list
            graph_document = graph_documents[0]
            
            # Extract nodes and relationships
            nodes = graph_document.nodes
            relationships = graph_document.relationships
            
            # Create nodes in Neo4j
            for node in nodes:
                session.run(
                    "CREATE (n:Node {id: $id, label: $label})",
                    id=node.id,
                    label=node.type
                )
            
            # Create relationships in Neo4j
            for relationship in relationships:
                session.run("""
                    MATCH (a:Node {id: $source_id}), (b:Node {id: $target_id})
                    CREATE (a)-[:RELATIONSHIP {type: $type}]->(b)
                """,
                source_id=relationship.source.id,  # Accessing the source node id
                target_id=relationship.target.id,  # Accessing the target node id
                type=relationship.type)
        else:
            print("Error: Expected a list with at least one GraphDocument but got:", type(graph_documents), graph_documents)

# Example usage with a list of graph documents
add_graph_to_neo4j(graph_documents)


NameError: name 'graph_documents' is not defined